In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Depending on your installation, you might have to adjust some settings to get these paths to work properly
import pipeline.get_data as get_data
import pipeline.preprocess as preprocess
import pipeline.base_models as base_models
from pipeline.base_models import load_base_modeled
import pipeline.learner as learner
from top_level_evaluation import evaluate_hyperparameters

import tensorflow as tf

In [ ]:
train_ds = load_base_modeled(base_models.vgg16_label, get_data.train_label)
valid_ds = load_base_modeled(base_models.vgg16_label, get_data.valid_label)
test_ds = load_base_modeled(base_models.vgg16_label, get_data.test_label)
input_shape = train_ds.element_spec[0].shape[1:]
print(input_shape)
data = (train_ds, valid_ds, test_ds)

Experiment 1: effect of varying epochs vs. steps_per_epoch while keeping the product constant

In [ ]:
n_samples = 30
evaluate_hyperparameters(learner.original_addon, data, 50, 32, 14, n_samples=n_samples)
evaluate_hyperparameters(learner.original_addon, data, 100, 32, 7, n_samples=n_samples)
evaluate_hyperparameters(learner.original_addon, data, 25, 32, 28, n_samples=n_samples)

No, not really. Lowering the number of epochs makes the loss spikes much more evident, though this just seems to be how the averaging turns out.

Experiment 2: effect of varying batch size while keeping all else constant

In [ ]:
evaluate_hyperparameters(learner.original_addon, data, 50, 32, 14, n_samples=n_samples)
evaluate_hyperparameters(learner.original_addon, data, 50, 16, 14, n_samples=n_samples)
evaluate_hyperparameters(learner.original_addon, data, 50, 64, 14, n_samples=n_samples)

Again, not much of an effect. Maybe slightly better with smaller batch sizes. We can just stick with the defaults then.

How many epochs?

In [ ]:
spe = tf.data.experimental.cardinality(train_ds).numpy()/preprocess.augmentation_reps
evaluate_hyperparameters(learner.original_addon, data, 50, None, spe, n_samples=3)
evaluate_hyperparameters(learner.original_addon, data, 500, None, spe, n_samples=3)
evaluate_hyperparameters(learner.original_addon, data, 100, None, spe, n_samples=3)

So it seems that 50 epochs is pretty good.